In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import torch as th

In [3]:
idx = th.load("../high_threshold_indices_4224000.pt")

/tmp/ipykernel_221757/2123677481.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  idx = th.load("../high_threshold_indices_4224000.pt")


In [4]:
from dlabutils import model_path
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

In [14]:
toks = tokenizer.apply_chat_template([{
    "role": "user",
    "content": "Hello, how are you?"
}])
print(tokenizer.decode(107))

<end_of_turn>


In [10]:
toks

[2, 106, 1645, 108, 4521, 235269, 1368, 708, 692, 235336, 107, 108]

In [16]:
one_init = th.load("../checkpoints/feature_scaler/L13-mu0.0e+00-lr1e-02/scaler_0_1.pt", map_location="cpu")
zero_init = th.load("../checkpoints/feature_scaler/L13-mu0.0e+00-lr1e-02-ZeroInit/scaler_0_1.pt", map_location="cpu")


/tmp/ipykernel_81867/766365240.py:1: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/tmp/ipykernel_81867/766365240.py:2: FutureWarning:

You are using `torch.load` with `weig

In [11]:
def act_func(x):
    return th.nn.functional.elu(x) + 1


In [32]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

scaled_values_one = act_func(one_init["scaler"]).numpy()
scaled_values_zero = act_func(zero_init["scaler"]).numpy()

# Create the figure
fig = go.Figure()

# Add histogram traces
fig.add_trace(go.Histogram(
    x=scaled_values_one,
    name='One Init',
    opacity=0.75,
    nbinsx=100
))

fig.add_trace(go.Histogram(
    x=scaled_values_zero,
    name='Zero Init',
    opacity=0.75,
    nbinsx=100
))

# Update layout
fig.update_layout(
    title='Distribution of Feature Scaling Values',
    xaxis_title='Scaling Factor',
    yaxis_title='Frequency',
    barmode='overlay',
    showlegend=True
)

fig.show()


In [26]:
# Create 2D histogram data
hist2d, x_edges, y_edges = np.histogram2d(
    scaled_values_one, 
    scaled_values_zero,
    bins=50
)

# Create the figure
fig = go.Figure(data=[
    go.Surface(
        x=x_edges[:-1],  # Use bin edges for x axis
        y=y_edges[:-1],  # Use bin edges for y axis
        z=hist2d.T,      # Transpose to match plotly's expected format
        colorscale='Viridis',
        colorbar=dict(title='Count')
    )
])

# Update layout
fig.update_layout(
    title='3D Distribution of Feature Scaling Values',
    scene=dict(
        xaxis_title='One Init Scaling Factor',
        yaxis_title='Zero Init Scaling Factor',
        zaxis_title='Count'
    )
)

fig.show()

In [30]:
# Create 2D histogram data
hist2d, x_edges, y_edges = np.histogram2d(
    scaled_values_one, 
    scaled_values_zero,
    bins=50
)

# Create the figure
fig = go.Figure(data=[
    go.Heatmap(
        x=x_edges[:-1],  # Use bin edges for x axis
        y=y_edges[:-1],  # Use bin edges for y axis
        z=np.log1p(hist2d.T),  # Log transform the counts, add 1 to handle zeros
        colorscale='Viridis',
        colorbar=dict(title='Log Count')
    )
])

# Update layout
fig.update_layout(
    title='2D Distribution of Feature Scaling Values',
    xaxis_title='One Init Scaling Factor',
    yaxis_title='Zero Init Scaling Factor',

)

fig.show()

In [31]:
# Create scatter plot
fig = go.Figure(data=[
    go.Scatter(
        x=scaled_values_one,
        y=scaled_values_zero,
        mode='markers',
        marker=dict(
            color='blue',
            opacity=0.5
        )
    )
])

# Update layout
fig.update_layout(
    title='Scatter Plot of Feature Scaling Values',
    xaxis_title='One Init Scaling Factor',
    yaxis_title='Zero Init Scaling Factor'
)

fig.show()
